In [1]:

import sys 
sys.path.append('../../')
import sys
sys.path.append('../../../')
sys.path.append('../../')
sys.path.append('../')

from core.conf_defaults import *
from core.tbal import * 
from core.auto_labeling import *
from core.run_lib import * 
from datasets.dataset_utils import * 
from utils.counting_utils import *  
from utils.common_utils import * 
from utils.vis_utils import *
from utils.logging_utils import * 
from  datasets.data_manager import * 
from omegaconf import OmegaConf


root_dir = '../../'
conf_dir = f'{root_dir}configs/tiny-imagenet-CLIP/'

base_conf_file = '{}/tiny_imagenet_CLIP_base_conf.yaml'.format(conf_dir)

logger      = get_logger(f'{root_dir}/temp/logs/al_tiny_imagenet_CLIP_test.log',stdout_redirect=True,level=logging.DEBUG)

conf = OmegaConf.load(base_conf_file)

set_defaults(conf)

conf['train_pts_query_conf']['max_num_train_pts'] = 10000

conf['data_conf']['compute_emb'] = False


conf['training_conf']['learning_rate']=0.1
conf['training_conf']['optimizer'] = 'sgd'
conf['training_conf']['batch_size']=64
conf['training_conf']['max_epochs']=50
conf['training_conf']['weight_decay']=0.001


In [2]:

set_seed(conf['random_seed'])

dm = DataManager(conf,logger)
len(dm.ds_std_train), len(dm.ds_std_val)

act_lbl = ActiveLearning(conf,dm,logger)

act_lbl.init()

out = act_lbl.run_al_loop()

auto_labeler = AutoLabeling(conf,dm,act_lbl.cur_clf,logger)
out = auto_labeler.run()
dm.get_auto_labeling_counts()


[01/02/2024 08:11:11 PM : INFO  : data_manag : ] : Loaded dataset tiny_imagenet_200_CLIP
[01/02/2024 08:11:11 PM : INFO  : data_manag : ] : Std train size: 100000 and Std. Val. Size:10000
[01/02/2024 08:11:11 PM : DEBUG : active_lea : ] : Stop Criterion max_num_train_pts
[01/02/2024 08:11:11 PM : DEBUG : active_lea : ] : Unlabeled Count In check_stop_criterion 100000
[01/02/2024 08:11:11 PM : DEBUG : active_lea : ] : cur_query_count= 0 and max_query_count=10000
[01/02/2024 08:11:11 PM : DEBUG : active_lea : ] : err_th 1.0 and val_err_threshold 0.0
[01/02/2024 08:11:11 PM : DEBUG : active_lea : ] : err_th 1.0 and val_err_threshold 0.0
[01/02/2024 08:11:11 PM : INFO  : active_lea : ] : ===========================================================================
[01/02/2024 08:11:11 PM : DEBUG : active_lea : ] : ========================= BEGIN EPOCH 0 ============================
[01/02/2024 08:11:11 PM : DEBUG : active_lea : ] : Number of unalabeled points  :100000
[01/02/2024 08:11:11 PM

{'auto_labeled_acc': 0.9571039966520193, 'coverage_1': 0.09558}

In [3]:

auto_labeler = AutoLabeling(conf,dm,act_lbl.cur_clf,logger)
out = auto_labeler.run()
dm.get_auto_labeling_counts()

[05/11/2023 10:43:53 PM : INFO  : auto_label : ] : ========================= Begin Auto-Labeling selective ==========================
[05/11/2023 10:43:53 PM : DEBUG : auto_label : ] : Auto Labeling Conf : {'method_name': 'selective', 'score_type': 'confidence', 'class_wise': 'independent', 'auto_label_err_threshold': 0.05, 'C_1': 0.05, 'ucb': 'sigma', 'fast': True}
[05/11/2023 10:43:53 PM : INFO  : auto_label : ] : Number of unlabeled points : 15000
[05/11/2023 10:43:54 PM : INFO  : auto_label : ] : Using number of validation points : 5000
[05/11/2023 10:43:54 PM : INFO  : auto_label : ] : Using Auto-Labeling Error Threshold = 0.05
[05/11/2023 10:43:54 PM : INFO  : auto_label : ] : Determining Thresholds : Class Wise : independent
[05/11/2023 10:43:54 PM : DEBUG : threshold_ : ] : MAX score = 0.5552254915237427, MIN score = 0.5552254319190979, delta = 2.9802321634825324e-12
[05/11/2023 10:43:54 PM : DEBUG : threshold_ : ] : C_1 = 0.05 UCB = sigma
[05/11/2023 10:43:56 PM : INFO  : thre

{'auto_labeled_acc': 0, 'coverage_1': 0.0, 'coverage_2': 0}